## Beautiful Demo

# Hello

In [1]:
from matplotlib import pyplot as plt
from read_data import training_validation
from processing import integral_image
from time import time
from random import choice

training, validation = training_validation()

sample_image = choice(training)

tp_ii = time()

ii = integral_image(sample_image)

print("time to preprocess image: {}".format(time() - tp_ii))


plt.imshow(sample_image)
plt.show()



reading csv: 2.947767972946167
formatting to numpy matrices: 4.278903007507324
picking random data: 0.2211160659790039
time to preprocess image: 0.017686843872070312


## Testing Naive Bayes 

In [2]:
from naive_bayes import naive_bayes
from numpy import zeros, arange
labels = zeros(len(training))
classifier = naive_bayes()

(n_of_imagest, widtht, heigtht) = training.shape
into_linear_featurest = training.reshape(n_of_imagest, widtht * heigtht)

classifier.train(into_linear_featurest, labels)

(n_of_imagesv, widthv, heightv) = validation.shape
into_linear_featuresv = validation.reshape(n_of_imagesv, widthv * heightv)

print(classifier.predict(into_linear_featuresv[0]))

[[[ 3990.9131       0.           0.       ...,     0.           0.           0.      ]
  [    0.        4186.603584     0.       ...,     0.           0.           0.      ]
  [    0.           0.        4452.681264 ...,     0.           0.           0.      ]
  ..., 
  [    0.           0.           0.       ...,  3251.208716     0.           0.      ]
  [    0.           0.           0.       ...,     0.        3224.776896
       0.      ]
  [    0.           0.           0.       ...,     0.           0.
    3099.335404]]]
[[ 161.57   158.304  156.644 ...,   90.222   92.152   93.814]]
[ 1.]
-1925.70851678
0
